In [3]:
import mechanicalsoup
import json
import hashlib
from bs4 import BeautifulSoup
import sys

BASE_URL = 'https://www.bbc.co.uk'

SCRAPING_SUFFIX = '/iplayer/a-z/'

browser = mechanicalsoup.StatefulBrowser()



In [4]:
# Dictionary to hold the data

program_data = {}

browser.open(BASE_URL + SCRAPING_SUFFIX)

selection = browser.get_current_page().find_all(
    'li', attrs={"class": "grid__item"})

navigation = browser.get_current_page().find('ul', attrs={'class': 'scrollable-nav__track'})

navigation_list = navigation.find_all('li')

navigation_list = [x.a['href'] for x in navigation_list if x.a is not None ]

print(navigation_list)

['/iplayer/a-z/a', '/iplayer/a-z/b', '/iplayer/a-z/c', '/iplayer/a-z/d', '/iplayer/a-z/e', '/iplayer/a-z/f', '/iplayer/a-z/g', '/iplayer/a-z/h', '/iplayer/a-z/i', '/iplayer/a-z/j', '/iplayer/a-z/k', '/iplayer/a-z/l', '/iplayer/a-z/m', '/iplayer/a-z/n', '/iplayer/a-z/o', '/iplayer/a-z/p', '/iplayer/a-z/q', '/iplayer/a-z/r', '/iplayer/a-z/s', '/iplayer/a-z/t', '/iplayer/a-z/u', '/iplayer/a-z/v', '/iplayer/a-z/w', '/iplayer/a-z/y', '/iplayer/a-z/z', '/iplayer/a-z/0-9']


From the initial wep page with the list of programs collect the title and synopsis. Can also grab the number of episodes. Collect link for further extraction

This section yields:
- Title
- Synopsis
- Episode Link



In [5]:
for item in selection:
    title = item.find('p', attrs={"class": "list-content-item__title"})
    synopsis = item.find('p', attrs={'class': 'list-content-item__synopsis'})
    num_episodes_available = item.find(
        'div', attrs={'class': 'list-content-item__sublabels'})
    link = item.find('a', href=True)['href']
    print('link: ',link)
    print('title: ', title.get_text())
    print('synopsis: ', synopsis.get_text())
    
    
    '''
    Follow link for each program item
    '''
    
    browser.open(BASE_URL + link)
    page = browser.get_current_page()

    episode_page_url = page.find('a', attrs={'class': 'lnk'}, text='Credits')
    if episode_page_url is None:
        episode_page_url = browser.get_current_page().find(
            'a', attrs={'class': 'lnk'}, text='Programme website')
    print('latest_episode_url: ',episode_page_url['href'])
    
    
    #navigate to episode page
    browser.open(BASE_URL + episode_page_url['href'])
    episode_page = browser.get_current_page()
    credits = episode_page.find('table', attrs={'class': 'table'})
    
    credits_dict = {}
    if credits is not None:
        for row in credits.find_all('tr'):
            person = row.find_all('span')
            if len(person) > 1:
                json_credits = [x.get_text() for x in person]
                credits_dict[json_credits[0]] = json_credits[1]
        print(credits_dict)

        
    
    similar = episode_page.find(
        'div', attrs={'class': 'footer__similar b-g-p component'})

    if similar is not None:
        sim = similar.find_all('div')
        genre_format = []

        for i in sim:
            genre_format.append(
                [[x.get_text(), x['href']] for x in i.find_all('a')])

        genre_dict={}
        prog_frmt_dict = {}
        genre = []
        prog_frmat = []

        for i in range(len(genre_format)):
            for j in range(len(genre_format[i])):
                if i == 0:
                    genre.append(genre_format[i][j])
                else:
                    prog_frmat.append(genre_format[i][j])

        print('genre: ', genre)
        print('format: ',prog_frmat)


    left_to_watch = episode_page.find(
        'div', attrs={'class': 'grid 1/3@bpw 1/4@bpe'})

    if left_to_watch is not None:
        left_to_watch_items = left_to_watch.find_all(
            'p', attrs={'class': 'episode-panel__meta'})

        if left_to_watch.find(
                'div', attrs={'class': "episode-panel__meta"}) is None:
            if left_to_watch_items[0].span is None:
                days_left = left_to_watch_items[0].get_text()
            else:
                days_left = left_to_watch_items[0].span.get_text()
            duration = left_to_watch_items[1].get_text()

        print('Left to watch: ', days_left,'duration: ' ,duration)
    
    
    
    long_synopsis = episode_page.find(
        'div', attrs={'class': 'synopsis-toggle__long'})

    if long_synopsis is not None:
        long_synopsis_paragraphs = [
            x.get_text() for x in long_synopsis.find_all('p')
        ]

        print('long_synopsis: ',long_synopsis_paragraphs)

    # Broadcast information:

    main_broadcast = episode_page.find(
        'div',
        attrs={
            'class':
            'grid 1/3@bpw2 1/3@bpe map__column map__column--2 map__column--last'
        })

    if main_broadcast is not None:
        date_last_aired = main_broadcast.find(
            'span',
            attrs={'class': 'broadcast-event__date text-base timezone--date'})
        time_last_aired = main_broadcast.find(
            'span', attrs={'class': 'timezone--time'})
        channel = main_broadcast.find(
            'div',
            attrs={
                'class':
                'programme__service box-link__elevated micro text--subtle'
            })
        if channel is not None:
            channel_text = channel.find('a').get_text()
        if channel is not None:
            channel_link = channel.find('a')['href']
        if date_last_aired is not None:
            date_last_aired = date_last_aired.get_text()
        if time_last_aired is not None:
            time_last_aired = time_last_aired.get_text()

        print('date: ', date_last_aired, 'Time: ', time_last_aired, 'Channel: ',
              channel_text, channel_link)
        
    # you may also like
    # navigate to : current page + /recommendations
    # scrape the same way as getting other episode data
    
    current_url = browser.get_url()
    
    browser.open(current_url + '/recommendations')
    
    recommend_page = browser.get_current_page()
    
    if recommend_page is not None:
    
        page_items = recommend_page.find('ol', attrs={'class': 'highlight-box-wrapper'})
        
        if page_items is not None: 
            list_items = page_items.find_all('li')
            for item in list_items:
                item_info = item.find('div', attrs={'class': 'programme__body'})
                link_1 = item_info.h4.a['href']
                link_2 = item_info.h4.a['resource']
                title = item_info.h4.a.get_text()
                synop = item_info.p.get_text()
                
                print('recommendations:', link_1, link_2, 'title: ',title, synop)
    
    
    episodes_link = episode_page.find(
        'a', attrs={
            'class': 'br-nav__link',
            'data-linktrack': 'nav_episodes'
        })
    
    if episodes_link is not None:
        episodes_link = episodes_link['href']

        print(episodes_link)
    
        browser.open(BASE_URL + episodes_link)
        
        
        episodes_page = browser.get_current_page()
        episodes_available_list = episodes_page.find(
            'div', attrs={'class': 'br-box-page programmes-page'})
        episodes_container_list = episodes_available_list.find_all(
            'div',
            attrs={
                'class':
                'programme programme--tv programme--episode block-link highlight-box--list br-keyline br-blocklink-page br-page-linkhover-onbg015--hover'
            })

        available_episodes = episodes_page.find('span', attrs={'class': 'hidden grid-visible@bpb2 grid-visible@bpw'})
        print('available episodes: ', available_episodes.get_text())
        
        for item in episodes_container_list:
            # link
            item_headder = item.find(
                'div', attrs={'class': 'cta cta__overlay'})
            item_link = item_headder.a['href']
            # time left
            item_time_left = item_headder.a['title']
            # title
            item_body = item.find('div', attrs={'class': 'programme__body'})
        
            print(item_link, item_time_left)
        
            if item_body is not None:
                # num of episodes
                try: 
                    episode_oneline_synopsis = item_body.p.get_text()
                    print('episode_oneline_synopsis: ' ,episode_oneline_synopsis)
                except:
                    pass
                try:
                    episode_no = item_body.p.abbr['title']
                    print('episode_no: ', episode_no)
                except:
                    pass
                try:
                    episode_title = item_body.find('span', attrs={'class': 'programme__title gamma'}).get_text()
                    print(episode_title)
                except:
                    pass
        
        
        episode_pagination = episode_page.find('ol', attrs={'class': 'nav nav--banner pagination delta'})
        
        print(episode_pagination)
        
        
        
        
        
    print('\n')

('link: ', '/iplayer/episode/p02f7d2x/abadas-sgi')
('title: ', u'Abadas')
('synopsis: ', u'Rhaglen animeiddio i blant ifanc yn canolbwyntio ar gyflwyno geiriau newydd. Animation ...')
('latest_episode_url: ', '/programmes/p02b4jth')
/programmes/p02b4jth/episodes
('available episodes: ', u'(7)')
('https://www.bbc.co.uk/iplayer/episode/p02f7d2x', u'25 o ddyddiau ar \xf4l i wylio (Sad 23 Chwefror 2019, 09:00)')
('episode_oneline_synopsis: ', u"\nMae'r Abadas yn chwarae ym mhyllau mwdlyd yr ardd pan ddaw Ben ar eu traws a'u gwahodd ...\n")
Sgi
('https://www.bbc.co.uk/iplayer/episode/p02f7cfh', u'23 o ddyddiau ar \xf4l i wylio (Iau 21 Chwefror 2019, 09:00)')
('episode_oneline_synopsis: ', u"\nMae gan Ben air Abada newydd sbon: 'melin wynt'. Ela gaiff ei dewis i fynd i chwilio am...\n")
Melin Wynt
('https://www.bbc.co.uk/iplayer/episode/p02dzdxj', u'18 o ddyddiau ar \xf4l i wylio (Sad 16 Chwefror 2019, 09:00)')
('episode_oneline_synopsis: ', u"\nTybed a fydd gair heddiw, 'anrheg' yn helpu El

('latest_episode_url: ', '/programmes/b08b7blz#credits')
{u'Lola': u'Indeyarna Donaldson-Holness', u'Bo': u'Mo Gaffney', u'Edina': u'Jennifer Saunders', u'Marshall': u'Christopher Ryan', u'Mother': u'June Whitfield', u'Writer': u'Jennifer Saunders', u'Lubliana': u'Marcia Warren', u'Patsy': u'Joanna Lumley', u'Director': u'Mandie Fletcher', u'Nick': u'Robert Webb', u'Magda': u'Kathy Burke', u'Claudia Bing': u'Celia Imrie', u'Saffy': u'Julia Sawalha', u'Stella McCartney': u'Stella McCartney', u'Bubble': u'Jane Horrocks'}
('genre: ', [[u'Comedy', '/programmes/genres/comedy'], [u'Sitcoms', '/programmes/genres/comedy/sitcoms']])
('format: ', [[u'Films', '/programmes/formats/films']])
('Left to watch: ', u'2 days left to watch', 'duration: ', u'\n1 hour, 31 minutes\n            ')
('long_synopsis: ', [u'Worried about her finances when her book fails to find a publisher, Patsy is given the tip that Kate Moss has left her PR company and will be looking for new representation. Things start to g

KeyboardInterrupt: 

In [ ]:
def iplayer_atoz_page_extractor():
    
    return 0

In [ ]:
def episode_page_extractor():
    
    return 0

In [ ]:
def programme_website_extractor():
    
    return 0

In [ ]:
def recommendation_extraction():
    
    return 0

In [ ]:
def episode_list_extraction():
    
    return 0

In [ ]:
def dictionary_builder():
    
    return 0


In [31]:
base_url = 'https://www.bbc.co.uk/programmes/b007y6k8/'
url = 'https://www.bbc.co.uk/programmes/b007y6k8/episodes/player'

def episode_pagination_test():
    episode_page = browser.get_current_page()
    pagination = episode_page.find('ol', attrs={'class': 'nav nav--banner pagination delta'})
    
    page_list = pagination.find_all('li', attrs={'class': 'pagination__page'})
    curr_url = browser.get_url()
    page_links = [x.a['href'] for x in page_list if x.a is not None]
    
    next_on_suffix = 'broadcasts/upcoming/'
    browser.open(base_url + next_on_suffix)
    
    next_on = browser.get_current_page()
    next_on_section = next_on.find('ol', attrs={'class':'highlight-box-wrapper'}) 
    
    next_up_dict = {}
    
    for item in next_on_section.find_all('li'):
        broadcast_info = item.find('div', attrs={'class':'programme__body programme__body--flush'})
        broadcast_info_tag = broadcast_info.find('div', attrs={'class': 'broadcast-event__time beta'})
        
        broadcast_date = broadcast_info_tag['title']
        broadcast_day = broadcast_info_tag.find('span', attrs={'class':'broadcast-event__date text-base timezone--date'}).get_text()
        broadcast_time = broadcast_info_tag.find('span',attrs = {'class': 'timezone--time'}).get_text()
        
        broadcast_channel = broadcast_info.find('div', attrs={'class': 'programme__service box-link__elevated micro text--subtle'})

        channel = broadcast_channel.a.get_text()
        channel_url = broadcast_channel.a['href']
        
        program_info = item.find('div', attrs={'class': 'grid 7/12 2/3@bpb2 3/4@bpw 5/6@bpw2 5/6@bpe'})
        program_title_info = program_info.a
        
        program_id = program_info.div['data-pid']
        program_link = program_title_info['href']
        program_title = program_title_info.find('span', attrs={'class': 'programme__title gamma'}).get_text()
        series = program_title_info.find('span', attrs={'class': 'programme__subtitle centi'}).get_text()
        program_synopsis = program_info.p.get_text()
        
        next_up_dict[program_id] = {'program_title': program_title.encode('utf-8'),
                                    'series': series.encode('utf-8'),
                                    'program_synopsis': program_synopsis.encode('utf-8').strip(),
                                    'program_link': program_link.encode('utf-8'),
                                    'channel': {'name': channel.encode('utf-8'),
                                              'link': channel_link.encode('utf-8')},
                                    'broadcast': {'date': broadcast_date.encode('utf-8'),
                                                 'day': broadcast_day.encode('utf-8'),
                                                 'time': broadcast_time.encode('utf-8')}}
        
        
    print(next_up_dict)
    
    print(page_links)
    
    episode_page_links = []
    for item in page_list:
        link = item.find('a')
        if link:
            episode_page_links.append(link['href'])
        
    print(episode_page_links)
    
    print(curr_url)
    
    for endpoint in episode_page_links:
        url = curr_url + endpoint
        
        browser.open(url)
        
        episodes_page = browser.get_current_page()
        episodes_available_list = episodes_page.find(
            'div', attrs={'class': 'br-box-page programmes-page'})
        episodes_container_list = episodes_available_list.find_all(
            'div',
            attrs={
                'class':
                'programme programme--tv programme--episode block-link highlight-box--list br-keyline br-blocklink-page br-page-linkhover-onbg015--hover'
            })

        for item in episodes_container_list:
            # link
            item_headder = item.find(
                'div', attrs={'class': 'cta cta__overlay'})
            item_link = item_headder.a['href']
            # time left
            item_time_left = item_headder.a['title']
            # title
            item_body = item.find('div', attrs={'class': 'programme__body'})
        
            print(item_link, item_time_left)
        
            if item_body is not None:
                # num of episodes
                try: 
                    episode_oneline_synopsis = item_body.p.get_text()
                    print('episode_oneline_synopsis: ' ,episode_oneline_synopsis)
                except:
                    pass
                try:
                    episode_no = item_body.p.abbr['title']
                    print('episode_no: ', episode_no)
                except:
                    pass
                try:
                    episode_title = item_body.find('span', attrs={'class': 'programme__title gamma'}).get_text()
                    print(episode_title)
                except:
                    pass
        

In [32]:
base_url = 'https://www.bbc.co.uk/programmes/b007y6k8/'
episodes_suffix = 'episodes/'
next_on_suffix = 'broadcasts/upcoming/'

browser.open(base_url + episodes_suffix)


episode_pagination_test()

{'b0c1tq4b': {'series': 'Series 22', 'broadcast': {'date': '28 Jan 2019', 'day': 'Today', 'time': '21:00'}, 'program_link': 'https://www.bbc.co.uk/programmes/b0c1tq4b', 'program_synopsis': 'A shocking incident on a suburban street brings back painful memories for Jack.', 'program_title': 'Deathmaker, Part One', 'channel': {'link': 'https://www.bbc.co.uk/bbctwo', 'name': 'BBC One'}}, 'b0c2ks57': {'series': 'Series 22', 'broadcast': {'date': '04 Feb 2019', 'day': 'Next Monday', 'time': '21:00'}, 'program_link': 'https://www.bbc.co.uk/programmes/b0c2ks57', 'program_synopsis': 'Nikki is humiliated in court when a barrister accuses her of making a serious error.', 'program_title': 'Betrayal, Part One', 'channel': {'link': 'https://www.bbc.co.uk/bbctwo', 'name': 'BBC One'}}, 'b0c2ksxl': {'series': 'Series 22', 'broadcast': {'date': '05 Feb 2019', 'day': 'Next Tuesday', 'time': '21:00'}, 'program_link': 'https://www.bbc.co.uk/programmes/b0c2ksxl', 'program_synopsis': 'When Thomas sides agains

('https://www.bbc.co.uk/iplayer/episode/b050m1sz', '10 months left to watch (Wed 11 December 2019, 09:00)')
('episode_oneline_synopsis: ', u"\n2/2 Nikki, Jack and Thomas focus in on Lizzie's movements on the night of Barge's murder.\n")
('episode_no: ', 'Episode 2 of 2')
Part 2
('https://www.bbc.co.uk/iplayer/episode/b050m0jj', '10 months left to watch (Wed 11 December 2019, 09:00)')
('episode_oneline_synopsis: ', u"\n1/2 A suspected paedophile is found murdered in a children's playground.\n")
('episode_no: ', 'Episode 1 of 2')
Part 1
('https://www.bbc.co.uk/iplayer/episode/b04ylrk4', '10 months left to watch (Wed 11 December 2019, 09:00)')
('episode_oneline_synopsis: ', u'\n2/2 Nikki helps high-flying DI Luke Nelson unearth painful childhood memories.\n')
('episode_no: ', 'Episode 2 of 2')
Part 2
('https://www.bbc.co.uk/iplayer/episode/b04ylr65', '10 months left to watch (Wed 11 December 2019, 09:00)')
('episode_oneline_synopsis: ', u'\n1/2 Nikki and Jack investigate a man killed by a